## Get Album from Spotify

In [1]:
# Test for album_lookup.py

from album_lookup import get_official_album_name, guess_album

# Example test: Try to look up a well-known track and check the album name
artist = "Iron Maiden"
title = "The Trooper"
expected_album = "Piece of Mind"

predicted_album = get_official_album_name(artist, title)
match = guess_album(artist, title)
print(f"Match details: {match}")
print(f"Artist: {artist}")
print(f"Title: {title}")
print(f"Expected Album: {expected_album}")
print(f"Predicted Album: {predicted_album}")

assert predicted_album is not None, "No album found"
assert expected_album.lower() in predicted_album.lower(), "Album lookup failed"

Match details: AlbumMatch(album='Piece of Mind', source='spotify', confidence=0.5777777777777778, album_type='album', raw_album='Piece of Mind (2015 Remaster)', release_date=datetime.datetime(1983, 1, 1, 0, 0), track_id='4OROzZUy6gOWN4UGQVaZMF', track_name='The Trooper - 2015 Remaster', title_score=0.6111111111111112, artist_score=1.0, popularity=74, flags=('reissue',))
Artist: Iron Maiden
Title: The Trooper
Expected Album: Piece of Mind
Predicted Album: Piece of Mind


In [2]:
from tqdm.notebook import tqdm
import pandas as pd

melodeath = "NeuralForge/playlists/Melodic Death Metal.csv"
classic_metal = "NeuralForge/playlists/Classic Metal.csv"
power_metal = "NeuralForge/playlists/Power Metal.csv"
folk_metal = "NeuralForge/playlists/Folk Metal.csv"
prog_metal = "NeuralForge/playlists/Prog Metal.csv"

# List of all playlist CSVs
playlist_files = [
    melodeath,
    # classic_metal,
    # power_metal,
    # folk_metal,
    # prog_metal,
]

# Concatenate all playlists into one DataFrame
df_all = pd.concat([pd.read_csv(f) for f in playlist_files], ignore_index=True)

correct = 0
total = 0

for i, row in tqdm(df_all.iterrows(), total=len(df_all)):
    artist = row["Artist"]
    title = row["Title"]
    official_album = row["Album"]
    predicted_album = get_official_album_name(artist, title)
    total += 1
    if predicted_album and str(predicted_album).lower() == str(official_album).lower():
        correct += 1
    else:
        print(f"\nMismatch for '{artist}' - '{title}':")
        print(f"\tOfficial: '{official_album}'")
        print(f"\tPredicted: '{predicted_album}'")

print(f"Correct: {correct}/{total} ({correct / total:.2%})")

  0%|          | 0/148 [00:00<?, ?it/s]


Mismatch for 'Ensiferum' - 'In My Sword I Trust':
	Official: 'Unsung Heroes'
	Predicted: 'Unsung Heroes (Japan Version)'

Mismatch for 'In Flames' - 'Moonshield':
	Official: 'The Jester Race'
	Predicted: 'The Jester Race (Black Ash-Inheritance Version)'

Mismatch for 'Insomnium' - 'Pale Morning Star':
	Official: 'Heart Like a Grave'
	Predicted: 'Pale Morning Star'

Mismatch for 'Insomnium' - 'Valediction':
	Official: 'Heart Like a Grave'
	Predicted: 'Valediction'

Mismatch for 'Mors Principium Est' - 'Reclaim the Sun':
	Official: 'Embers of a Dying World'
	Predicted: 'Embers of a Dying World (Bonus Version)'

Mismatch for 'Amorphis' - 'Bones':
	Official: 'Bones'
	Predicted: 'Borderland'

Mismatch for 'Amorphis' - 'Dancing Shadow':
	Official: 'Dancing Shadow'
	Predicted: 'Borderland'

Mismatch for 'Amorphis' - 'Light And Shadow':
	Official: 'Light And Shadow'
	Predicted: 'Borderland'

Mismatch for 'Mors Principium Est' - 'All Life is Evil':
	Official: 'All Life is Evil'
	Predicted: 'Da